In [1]:
using Plots
using Random

include("acquisition_functions.jl")
include("gp_utils.jl")
include("invariant_gps.jl")
include("plot_utils.jl")
include("kernel_objective_function.jl")

seed = 2024;
Random.seed!(seed);

In [2]:
# Function drawn from 2D symmetric GP
d = 2
bounds = [(0.0, 1.0) for i in 1:d]
θ_true = (
    l=0.2,
    σ_f=1.0,
    σ_n=0.1,
)
xi_range = range(0.0, 1.0, length=100)
f = build_latent_function(build_permutationinvariantmatern52_gp, θ_true, 64, bounds, 17)
contourf(xi_range, xi_range, (x, y) -> f([x, y]), levels=32, color=:viridis, cbar=true, size=(300, 300), title="Ground truth")

InterruptException: InterruptException:

In [3]:
# Preallocate arrays for the samples
n_steps = 256
observed_x = Vector{Vector{Float64}}(undef, n_steps)
observed_y = Vector{Float64}(undef, n_steps)

# Generate the initial observation
observed_x[1] = [
    rand(Uniform(lower, upper))
    for (lower, upper) in bounds
]
observed_y[1] = f(observed_x[1])
println("Initial observation: ", observed_x[1], " -> ", observed_y[1])
plot_2d_function_with_observations(f, observed_x[1:1])

UndefVarError: UndefVarError: `f` not defined

In [4]:
# Initial GP fitting
# Prior on the GP hyperparameters
θ_0 = (
    σ_f=1.0,
    l=[1.0 for _ in 1:d],
    σ_n=0.1
)

# Build the GP
gp = get_posterior_gp(build_permutationinvariantmatern52_gp, observed_x[1:1], observed_y[1:1], θ_0; optimise_hyperparameters=true)
plot_2d_gp_with_observations(gp, observed_x[1:1])
savefig("mvr_1.png")

┌ Warning: No strict ticks found
└ @ PlotUtils /home/theo/.julia/packages/PlotUtils/jEGKP/src/ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils /home/theo/.julia/packages/PlotUtils/jEGKP/src/ticks.jl:191


"/home/theo/documents/ucl/bayesopt_under_invariance/mvr_1.png"

In [5]:
for i in 2:n_steps
    # Generate the next observation
    x_next = maximise_acqf(gp, mvr, bounds, 256)
    observed_x[i] = x_next
    observed_y[i] = f(x_next)
    println("[$i/$n_steps]: ", observed_x[i], " -> ", observed_y[i])
    plot_2d_function_with_observations(f, observed_x[1:i])

    # Update the GP
    gp = get_posterior_gp(build_permutationinvariantmatern52_gp, observed_x[1:i], observed_y[1:i], θ_0; optimise_hyperparameters=true)
    plot_2d_gp_with_observations(gp, observed_x[1:i])
    savefig("mvr_$(i).png")
end

UndefVarError: UndefVarError: `f` not defined